In [ ]:
import os
import numpy as np
import pandas as pd
from utility_functions import get_f1_score, add_row_to_csv

In [ ]:
dataset = "epigenomic"
# model = "linear"
# model = 'MMIT'
# model = 'aft_xgboost'
model = 'mlp'

In [ ]:
# get number of folds
fold_df = pd.read_csv('training_data/' + dataset + '/folds.csv')
n_folds = np.unique(fold_df['fold']).__len__()

In [ ]:
# prediction_folder_path = '1.' + model + '/predictions/'
# prediction_folder_path = "1." + model + '/linear_L1reg/predictions/'
prediction_folder_path = model + '/predictions/'

# List all files in the folder
file_names = os.listdir(prediction_folder_path)

def extract_between_dots(name):
    parts = name.rsplit('.')
    return parts[-2]

n_features_list = [extract_between_dots(name) for name in file_names]
n_features_list = np.int64(np.unique(n_features_list))

In [ ]:
def get_F1_from(dataset, test_fold, n_feature):
    evaluation_df = pd.read_csv('training_data/' + dataset + '/evaluation.csv')
    fold_df = pd.read_csv('training_data/' + dataset + '/folds.csv')
    eval_df = evaluation_df[evaluation_df['sequenceID'].isin(fold_df[fold_df['fold'] == test_fold]['sequenceID'])]
    lldas_df = pd.read_csv(model + '/predictions/' + dataset + '.' + str(test_fold) + '.' + str(n_feature) + '.csv')
    # lldas_df = pd.read_csv('1.' + model + '/linear_unreg/predictions/' + '.' + dataset + '.' + str(test_fold) + '.' + str(n_feature) + '.csv')
    # lldas_df = pd.read_csv('1.' + model + '/linear_L1reg/predictions/' + '.' + dataset + '.' + str(test_fold) + '.' + str(n_feature) + '.csv')
    # lldas_df = pd.read_csv('1.' + model + '/predictions/' + '.' + dataset + '.' + str(test_fold) + '.' + str(n_feature) + '.csv')
    return get_f1_score(eval_df, lldas_df)

In [ ]:
for test_fold in range(1, n_folds+1):
    for n_feature in n_features_list:
        F1 = get_F1_from(dataset, test_fold, n_feature)
        # print(dataset, test_fold, n_feature, acc)
        add_row_to_csv('F1_score_csvs/' + dataset + '.csv', 
                ['method', 'fold', 'F1'],
                [model.lower() + '.' + str(n_feature), test_fold, F1])